<a href="https://colab.research.google.com/github/yukismd/LLM/blob/main/test_yukismd_JapaneseQuizChatbot_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

モデル： https://huggingface.co/yukismd/JapaneseQuizChatbot_v1

In [1]:
!nvidia-smi

Fri Jul 28 02:56:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers==4.28.1
!pip install accelerate==0.18.0
!pip install torch==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import torch
from transformers import pipeline

generate_text = pipeline(
    model="yukismd/JapaneseQuizChatbot_v1",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    use_fast=True,
    device_map={"": "cuda:0"},
)

res = generate_text(
    "日本で一番高い山は富士山ですが、二番目に高い山は？",
    min_new_tokens=2,
    max_new_tokens=256,
    do_sample=False,
    num_beams=2,
    temperature=float(0.3),
    repetition_penalty=float(1.2),
    renormalize_logits=True
)
print(res[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


北岳


In [8]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/yukismd/LLM/main/model_test/data/quiz.csv')
print(df.shape)
df.head()

(25, 4)


,no,Source,Quiz,Answer
0,1,https://blog-knowledgequiz.com/betamon/,火を神聖視することから「拝火教」とも呼ばれる、古代ペルシアを起源とする宗教の名前はなんでしょう？,ゾロアスター教
1,2,https://blog-knowledgequiz.com/betamon/,「なぜ山に登るのか？」と聞かれて「そこに山があるから」と答えた逸話で知られるイギリスの登山家...,ジョージ・マロリー
2,3,https://blog-knowledgequiz.com/betamon/,漢数字の「百」から「一」を引くということから、99歳のお祝いのことを何というでしょう？,白寿
3,4,https://blog-knowledgequiz.com/betamon/,陸上の十種競技で、最初に行われるのは100メートル走ですが、最後に行われるのは何でしょう？,1500m走
4,5,https://blog-knowledgequiz.com/betamon/,県の木をイチョウとしている都道府県は、東京都、大阪府とどこでしょう？,神奈川県


In [10]:
def answer_quiz(quiz):
  res = generate_text(
    quiz,
    min_new_tokens=2,
    max_new_tokens=256,
    do_sample=False,
    num_beams=2,
    temperature=float(0.3),
    repetition_penalty=float(1.2),
    renormalize_logits=True
  )
  return res[0]["generated_text"]

In [15]:
answer = []
for q in df['Quiz']:
  a = answer_quiz(q)
  answer.append(a)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token

In [16]:
answer

['ゾロアスター教',
 'エドモンド・ヒラリー',
 '白寿',
 '砲丸投',
 '沖縄県。',
 'オーストリア共和国',
 'スプーン・スプーン',
 '77億人',
 '斥候',
 'さいたま市\nさいたま市',
 'マーガリン',
 '九龍半島',
 '金曜カレー',
 'オームの法則',
 '肝臓。',
 '港区。',
 '蟹工船',
 'ヘリウムガス',
 'サボタージュ',
 'ペニシリン',
 '剛田',
 '腕立て伏せ',
 '日本海溝',
 '体育教師',
 'はさみ']

In [17]:
len(answer)

25

In [18]:
df['Answer_JapaneseQuizChatbot_v1'] = answer

In [19]:
df

,no,Source,Quiz,Answer,Answer_JapaneseQuizChatbot_v1
0,1,https://blog-knowledgequiz.com/betamon/,火を神聖視することから「拝火教」とも呼ばれる、古代ペルシアを起源とする宗教の名前はなんでしょう？,ゾロアスター教,ゾロアスター教
1,2,https://blog-knowledgequiz.com/betamon/,「なぜ山に登るのか？」と聞かれて「そこに山があるから」と答えた逸話で知られるイギリスの登山家...,ジョージ・マロリー,エドモンド・ヒラリー
2,3,https://blog-knowledgequiz.com/betamon/,漢数字の「百」から「一」を引くということから、99歳のお祝いのことを何というでしょう？,白寿,白寿
3,4,https://blog-knowledgequiz.com/betamon/,陸上の十種競技で、最初に行われるのは100メートル走ですが、最後に行われるのは何でしょう？,1500m走,砲丸投
4,5,https://blog-knowledgequiz.com/betamon/,県の木をイチョウとしている都道府県は、東京都、大阪府とどこでしょう？,神奈川県,沖縄県。
5,6,https://blog-knowledgequiz.com/betamon/,フィギュアスケートのルッツジャンプに名前を残す「アロイス・ルッツ」の出身国はどこでしょう？,オーストリア,オーストリア共和国
6,7,https://blog-knowledgequiz.com/betamon/,ポテトサラダやアイスクリームをすくうのに用いる、半球状の器に取っ手がついた器具を何というでしょう？,ディッシャー,スプーン・スプーン
7,8,https://bonjin-ultra.com/mondai8.htm,2022年現在における世界の人口は、約何人？,80億人,77億人
8,9,https://bonjin-ultra.com/mondai8.htm,ボーイスカウトの「スカウト」とはどういう意味？,斥候,斥候
9,10,https://bonjin-ultra.com/mondai8.htm,\t浦和市・大宮市・与野市が合併して誕生した市は？,さいたま市,さいたま市\nさいたま市


In [20]:
df.to_csv('res_JapaneseQuizChatbot_v1.csv', index=False)